In [2]:
import pandas as pd
from utils import *
import re
regexp = re.compile(r'фоп|фізична особа підприємець')

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_9876/1131316670.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
test_df = pd.read_excel('Data/Auxillary/Список закладів.xlsx', skiprows=1)
test_df['legal_entity_edrpou'] = test_df['legal_entity_edrpou'].apply(lambda x: x if (x is pd.NA or x == 'ФОП') else str(x).rjust(8, '0'))
test_df = test_df[test_df['care_type']=='Первинна']
test_df

,legal_entity_id,legal_entity_name,legal_entity_edrpou,care_type,property_type,legal_entity_email,legal_entity_website,legal_entity_phone,legal_entity_owner_name,registration_area,registration_settlement,registration_address,lat,lng
2,6e6b9d4e-231d-4b6a-a8d7-081f055fe8d4,КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО УСАТІВСЬК...,38617509,Первинна,Комунальна,linarudoman@gmail.com,NaN,+380485225112,Рудоманенко Олена Вікторівна,ОДЕСЬКА,УСАТОВЕ,"Україна, 67663, Одеська обл., Одеський р-н, се...",46.532867,30.657932
3,c2b00dbc-bbb9-4b8c-a4df-7a409e5c575f,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ...",37934309,Первинна,Комунальна,ugledariao@gmail.com,NaN,+380627364093,Сосюра Наталія Миколаївна,ДОНЕЦЬКА,ВУГЛЕДАР,"Україна, 85670, Донецька обл., місто Вугледар,...",47.775308,37.239382
4,a2e144ce-0307-4c6e-bdb9-6186d519a531,МІСЬКО ВІТА ВАСИЛІВНА,ФОП,Первинна,ФОП,vt.klinik@gmail.com,NaN,+380506906522,Місько Віта Василівна,ЧЕРНІВЕЦЬКА,ЗАСТАВНА,"Україна, 59400, Чернівецька обл., Заставнівськ...",48.531439,25.833238
5,ba94b288-1b59-4f58-9f0c-117b3f8ac7f9,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""МЕДИЧ...",37691162,Первинна,Приватна (без ФОП),moe_zdorovie@ukr.net,NaN,+380974631203,Хмільовий Дмитро Андрійович,ВІННИЦЬКА,ВІННИЦЯ(З),"Україна, 21000, Вінницька обл., місто Вінниця(...",49.236732,28.412584
6,eb1b47d6-d116-446f-bfd4-d000884574ca,ЯКИМЧУК НАТАЛІЯ ВАЛЕРІЇВНА,ФОП,Первинна,ФОП,n.yakimchuk@homedoctor.in.ua,homedoctor.in.ua,+380637000640,Якимчук Наталія Валеріївна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.578154,30.799197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,8fdb1613-3c19-491c-bdbb-7a124cb5f8c7,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ДОБРОПІЛ...",37755220,Первинна,Комунальна,cpmsd.dobr@ukr.net,https://dcpmsd.dn.ua,+380627725303,Козенятко Ірина Іванівна,ДОНЕЦЬКА,ДОБРОПІЛЛЯ,"Україна, 85000, Донецька обл., місто Добропілл...",48.462386,37.079921
4115,a065e232-e36f-408f-a9e0-60243a584c00,КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО МИКОЛАЇВ...,38458175,Первинна,Комунальна,cpmsd7_office@ukr.net,https://cpmsd7.lic.org.ua,"+380678073761, +380512640816",Власова Олена Миколаївна,МИКОЛАЇВСЬКА,МИКОЛАЇВ,"Україна, 54051, Миколаївська обл., місто Микол...",46.856015,32.017890
4116,5c6d9036-7a55-40de-aa86-b5c355303bbb,ПАВЛЕНКО НАТАЛІЯ ОЛЕКСІЇВНА,ФОП,Первинна,ФОП,nata.pa1976@gmail.com,NaN,+380956281094,Павленко Наталія Олексіївна,ХАРКІВСЬКА,БОРОВА,"Україна, 63801, Харківська обл., Ізюмський р-н...",49.374251,37.623014
4119,cb19377d-8c28-4f75-b1b4-d6cf243c0622,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""УКРАЇ...",31066166,Первинна,Приватна (без ФОП),admin@unicam.com.ua,unicam.com.ua,+380500303034,Білецький Павло Степанович,СУМСЬКА,СУМИ,"Україна, 40034, Сумська обл., місто Суми, ПРОС...",50.914314,34.833996


In [4]:
test_edrdou_set = set(test_df['legal_entity_edrpou'].unique())

test_df['legal_entity_edrpou'].str.len().value_counts()

legal_entity_edrpou
8    1660
3     819
Name: count, dtype: int64

##### Підвантажити і обробити дані з таблиці залишків

In [5]:
df = pd.read_excel('Data/ВСЬОГО – 2024-02-23.xlsx', sheet_name='Залишки поточні', dtype={' код ЄДРПОУ': str}, engine='openpyxl')
df.columns = df.columns.str.strip()
df.rename(columns={'Серія препарту': 'Серія препарату'}, inplace=True)
# ———————————————————————————— Визначити назву вакцини за серією препарату і переписати назви у вхідних даних ————————————————————————————
series_df = df.groupby(['Серія препарату'])['Міжнародна непатентована назва'].apply(lambda vaccines: [[vaccine, len(vaccines[vaccines==vaccine])] for vaccine in vaccines.unique() if not pd.isna(vaccine)]).reset_index()

# Залишити тільки серії, де є хоча б одна назва препарату
series_df = series_df[~(series_df['Міжнародна непатентована назва'].apply(lambda vaccines: len(vaccines))==0)]

# Обрати назву препарату, яка найчастіше зустрічається в серії
overwritings = series_df.apply(lambda row: max(row['Міжнародна непатентована назва'], key=lambda pair: pair[1])[0], axis=1)
series_df['Назва препарату'] = overwritings

series_df = series_df[['Серія препарату', 'Назва препарату']].set_index('Серія препарату')

df['Міжнародна непатентована назва'] = df[['Серія препарату', 'Міжнародна непатентована назва']].apply(lambda row: series_df.to_dict()['Назва препарату'].get(row['Серія препарату'], row['Міжнародна непатентована назва']), axis=1)
# ————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
df['Міжнародна непатентована назва'] = df['Міжнародна непатентована назва'].replace(vaccine_shorts)

# Remove vaccines with no Facility name and EDRPOU
facilitiles_with_no_edrpou_set = set(df[(df['код ЄДРПОУ']=='') | (df['код ЄДРПОУ'].isna())]['Заклад'].dropna().unique()) - set(df[~((df['код ЄДРПОУ']=='') | (df['код ЄДРПОУ'].isna()))]['Заклад'].dropna().unique())
facilitiles_with_no_edrpou_df = df[df['Заклад'].isin(facilitiles_with_no_edrpou_set)]
df = df[ ~((df['Заклад'].isna() | (df['Заклад'].str.len() == 0)) & (df['код ЄДРПОУ'].isna() | (df['код ЄДРПОУ'].str.len() == 0))) ]

df['код ЄДРПОУ'] = df['код ЄДРПОУ'].str.rjust(8, '0')
df['код ЄДРПОУ'].fillna('', inplace=True)
edrpou_series = df['код ЄДРПОУ']
name_series = df['Заклад']

df['property_type'] = df['Заклад'].apply(lambda x: 'ФОП' if regexp.search(x.lower()) else 'Заклад')

df

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_9876/1582615033.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['код ЄДРПОУ'].fillna('', inplace=True)


,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Торгівельна назва,Виробник,Термін придатності,Джерело фінансування,Кількість доз,дата оновлення,Регіон
0,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",1141M019A,ХІБ,ВАКЦИНА КОН`ЮГОВАНА ДЛЯ ПРОФІЛАКТИКИ ЗАХВОРЮВА...,"СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2024-05-21 00:00:00,Державний бюджет 2021,54,2024-02-23,Вінницька область
1,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",X007788,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26 00:00:00,Гуманітарна допомога,6,2024-02-23,Вінницька область
2,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",D2708,АДП-М,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",ББ-НЦІПХЛтд. Болгарія,2025-11-30 00:00:00,Державний бюджет 2021,113,2024-02-23,Вінницька область
3,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",2249001B,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V.,2025-12-08 00:00:00,Гуманітарна допомога,13,2024-02-23,Вінницька область
4,35527334,"КНП ""ЦПМСД №1"" м. Вінниця",62С22009А,Сказ,"ІНДІРАБ ВАКЦИНА АНТИРАБІЧНА ОЧИЩЕНА, ІНАКТИВОВ...","ТОВ ""Фарма Лайф"", Україна",2025-10-31 00:00:00,Державний бюджет 2021,7,2024-02-23,Вінницька область
...,...,...,...,...,...,...,...,...,...,...,...
18386,3016503465,ФОП Мирошниченко,2310002A,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V. Netherlands,2026-02-14 00:00:00,Гуманітарна допомога,5,2024-02-23,Запорізька область
18387,3016503465,ФОП Мирошниченко,2863X026B,Пента,"Diphtheria, Tetanus, WholeCell Pertussis, Hepa...","СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2026-01-31 00:00:00,Державний бюджет 2022,10,2024-02-23,Запорізька область
18388,3016503465,ФОП Мирошниченко,68D23040А,ОПВ,"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bharat Biotech Int.Ltd, Індія",2024-08-01 00:00:00,Гуманітарна допомога,10,2024-02-23,Запорізька область
18389,38732602,"ТОВ ""ОНКОЛАЙФ""",68D23040А,ОПВ,"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bharat Biotech Int.Ltd, Індія",2024-08-01 00:00:00,Гуманітарна допомога,20,2024-02-23,Запорізька область


In [6]:
df.dropna(subset=['Заклад'], inplace=True)
df[['код ЄДРПОУ', 'property_type']].drop_duplicates()['property_type'].value_counts()

property_type
Заклад    1531
ФОП        346
Name: count, dtype: int64

##### Знайти заклади, які мають багато назв на один ЄДРПОУ

In [8]:
multiple_names_edrpou = df.groupby('код ЄДРПОУ')['Заклад'].apply(lambda x: list(set(x)))
multiple_names_edrpou = multiple_names_edrpou[multiple_names_edrpou.apply(lambda x: len(x)>1)]
multiple_names_edrpou = multiple_names_edrpou.loc[multiple_names_edrpou.index.str.len() > 0].index

multiple_names_df = df[df['код ЄДРПОУ'].isin(multiple_names_edrpou)][['код ЄДРПОУ', 'Регіон', 'Заклад']]
multiple_names_df = multiple_names_df.groupby('код ЄДРПОУ')[['Заклад', 'Регіон']].apply(lambda x: pd.Series(["\n".join(list(x['Заклад'])), x['Регіон'].iloc[0]], index=['Заклад', 'Регіон']))
multiple_names_df

,Заклад,Регіон
код ЄДРПОУ,,
01990756,"КНП""БЛІЛ КМР""\nКНП""БЛІЛ КМР""\nКНП""БЛІЛ КМР""\nК...",Донецька область
01993486,"КНП''Тисменицька міська лікарня""Тисменицької м...",Івано-Франківська область
01995290,"КНП ""Петрівська Центральна лікарня"" Петрівсько...",Кіровоградська область
01998354,"КНП ""Баштанська БПЛ""\nКНП ""Баштанська БПЛ""\nКН...",Миколаївська область
01998785,"КНП ""Кодимська лікарня"" Кодимської міської рад...",Одеська область
...,...,...
43762272,ТОВ МЦ Лікарська династія\nТОВ МЦ Лікарська ди...,Київська область
43778764,"ТОВ ""Медикал плюс""\nТОВ ""Медикал плюс""\nТОВ ""М...",Волинська область
43858467,"ТОВ ""Первинка - 2""\nТОВ ""Первинка - 2""\nТОВ ""П...",Дніпропетровська область


In [9]:
facilitiles_with_no_edrpou_df.groupby("Регіон").apply(lambda x: pd.Series(["\n".join(list(set(x['Заклад'])))], index=['Заклад']))

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_9876/1258525886.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  facilitiles_with_no_edrpou_df.groupby("Регіон").apply(lambda x: pd.Series(["\n".join(list(set(x['Заклад'])))], index=['Заклад']))


,Заклад
Регіон,
Івано-Франківська область,"КНП""КУТСЬКА МІСЬКА ЛІКАРНЯ""КУТСЬКОЇ СЕЛИЩНОЇ ..."
Дніпропетровська область,"ФОП Каліта Д.К.\nФОП Пасько Г.І.\nТОВ ""Перша п..."
Житомирська область,"КНП "" АЗПСМ"" Квітневої сільської ради"
Закарпатська область,ФОП ТОВТ\nФОП СЕНИНА\nКОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ...
Львівська область,КНП «ЛЬВІВСЬКЕ ТЕРИТОРІАЛЬНЕ МЕДИЧНЕ ОБ'ЄДНАНН...
Одеська область,"КНП ""Ширяївська ЦРЛ"""
Полтавська область,Фізична особа-підприємець Шупейко Анастасія Іг...
Рівненська область,"КНП ""Костопільська багатопрофільна лікарня інт..."
Сумська область,"ФОП Рогаль Л.І.\nКНП ""Краснопільський ЦПМСД"" К..."


In [10]:
df[ (df['код ЄДРПОУ'].isin({el for el in edrpou_series.unique() if len(el)!=8})) & ~(df['Заклад'].fillna('').str.lower().str.contains("фоп")) ][['Регіон', 'код ЄДРПОУ', 'Заклад']].drop_duplicates('код ЄДРПОУ').to_excel(
    'Misc/Заклади з неправильними ЄДРПОУ.xlsx', index=False
)

##### Які довжини у кодів ЄДРПОУ у таблиці Стариченка?
> ###### Дозволяє відстежити наявність записів типу "ФОП"

In [11]:
df['код ЄДРПОУ'].drop_duplicates().str.len().value_counts()

код ЄДРПОУ
8     1524
10     347
9        4
0        1
Name: count, dtype: int64

##### Скільки:
* Унікальних ЄДРПОУ (таблиця Стариченка)
* Унікальних назв (таблиця Стариченка)
* Унікальних ЄДРПОУ (перевірочна таблиця)
* Спільних ЄДРПОУ

In [12]:
len(set(edrpou_series)), len(set(name_series)), len(test_edrdou_set), len(set(edrpou_series) & test_edrdou_set)

(1876, 2224, 1661, 1064)

In [13]:
df['Міжнародна непатентована назва'].value_counts()

Міжнародна непатентована назва
КПК         2313
ГепВ        2278
ІПВ         2162
АДП-М       1769
Пента       1736
АКДП        1665
ОПВ         1628
ХІБ         1491
АДП         1261
БЦЖ          900
Сказ         796
ГепВ-Дор     367
Правець        8
Name: count, dtype: int64

In [14]:
# Drop all the duplicate lines with the same EDRPOU (different lines usually represent different responsive persons in the same facility)
facilities_only_test_df = test_df.drop_duplicates('legal_entity_edrpou')
facilities_only_test_df = facilities_only_test_df[facilities_only_test_df['legal_entity_edrpou']!="ФОП"]

result = pd.DataFrame(columns=["Міжнародна непатентована назва", "registration_area", "legal_entity_name", "код ЄДРПОУ", "legal_entity_phone", "legal_entity_email"])
for vaccine, data in df.groupby('Міжнародна непатентована назва'):
    # Pick the data for one specific vaccine and outer join it with the dataset of all the facilities that should have it
    facilities_reporing_vaccine = data.join(facilities_only_test_df.set_index('legal_entity_edrpou'), on='код ЄДРПОУ', how='outer', rsuffix='_test')
    
    # Now, if the vaccine_name field is empty, it means that the facility does not report this vaccine (otherwise it would have been joined by the EDRPOU)
    data_to_add = facilities_reporing_vaccine[facilities_reporing_vaccine['Міжнародна непатентована назва'].isna()][[
        'Міжнародна непатентована назва', "registration_area", "legal_entity_name", "код ЄДРПОУ", "legal_entity_phone", "legal_entity_email"
    ]]
    
    # Fill the vaccine name and add the data to the result
    data_to_add['Міжнародна непатентована назва'] = vaccine
    result = pd.concat([result, data_to_add], ignore_index=True, axis=0)

with pd.ExcelWriter('Misc/Заклади, які не звітують про вакцину.xlsx') as writer:
    for region, data in result.groupby('registration_area'):
        data.to_excel(writer, sheet_name=region, index=False)

    ignorant_facilities = result[~result['Міжнародна непатентована назва'].isin(['ГепВ-Дор', 'Правець', 'ДПКГП'])]

    ignorant_facilities_names = ignorant_facilities.groupby('код ЄДРПОУ')['Міжнародна непатентована назва'].nunique() == 11
    # in pd.Series leave only those index values that are True
    ignorant_facilities_names = ignorant_facilities_names.loc[lambda x: x].index

    ignorant_facilities = ignorant_facilities[ignorant_facilities['код ЄДРПОУ'].isin(ignorant_facilities_names)].drop('Міжнародна непатентована назва', axis=1).drop_duplicates().sort_values('registration_area')
    ignorant_facilities.to_excel(writer, sheet_name="Заклади, які не звітують про жодну вакцину", index=False)
    
    multiple_names_edrpou = df.groupby('код ЄДРПОУ')['Заклад'].apply(lambda x: list(set(x)))
    multiple_names_edrpou = multiple_names_edrpou[multiple_names_edrpou.apply(lambda x: len(x)>1)]
    multiple_names_edrpou = multiple_names_edrpou.loc[multiple_names_edrpou.index.str.len() > 0].index
    
    multiple_names_df = df[df['код ЄДРПОУ'].isin(multiple_names_edrpou)][['код ЄДРПОУ', 'Регіон', 'Заклад']]
    multiple_names_df = multiple_names_df.groupby('код ЄДРПОУ')[['Заклад', 'Регіон']].apply(lambda x: pd.Series([ x['Регіон'].iloc[0], "\n".join(list(set(x['Заклад'])))], index=['Регіон', 'Заклад'])).reset_index()
    
    multiple_names_df.to_excel(writer, sheet_name="Заклади з однаковим ЄДРПОУ|ІПН", index=False)
    
    facilitiles_with_no_edrpou_df.groupby("Регіон").apply(lambda x: pd.Series(["\n".join(list(set(x['Заклад'])))], index=['Заклад'])).reset_index().to_excel(writer, sheet_name="Заклади без ЄДРПОУ", index=False)
    

/Users/andrewlickbrekht/.local/share/virtualenvs/Vaccine_Administration_Analysis-tY9_woho/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_9876/3481180725.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  facilitiles_with_no_edrpou_df.groupby("Регіон").apply(lambda x: pd.Series(["\n".join(list(set(x['Заклад'])))], index=['Заклад'])).reset_index().to_excel(writer, sheet_name="Заклади без ЄДРПОУ", index=False)


##### Знайти усіх ФОПів, які звітують у таблицю Стариченка і містяться у файлі від ЦГЗ


In [15]:
fops_df = df[df['Заклад'].str.lower().str.contains('фоп')]
fops_df.loc[:, 'Регіон'] = fops_df['Регіон'].str.lower().str.replace('область', '').str.replace('м. київ', 'м.київ').str.strip()
fops_df.loc[:, 'Прізвище'] = fops_df['Заклад'].str.lower().str.replace(regexp, '', regex=True).str.strip().str.split(' ').str[0] + '+' + fops_df['Регіон']
fops_df

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_9876/3825026174.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fops_df.loc[:, 'Прізвище'] = fops_df['Заклад'].str.lower().str.replace(regexp, '', regex=True).str.strip().str.split(' ').str[0] + '+' + fops_df['Регіон']


,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Торгівельна назва,Виробник,Термін придатності,Джерело фінансування,Кількість доз,дата оновлення,Регіон,property_type,Прізвище
816,3302705015,ФОП Мельник,1141M019A,ХІБ,ВАКЦИНА КОН`ЮГОВАНА ДЛЯ ПРОФІЛАКТИКИ ЗАХВОРЮВА...,"СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2024-05-21 00:00:00,Державний бюджет 2021,9,2024-02-23,вінницька,ФОП,мельник+вінницька
817,3302705015,ФОП Мельник,2249001B,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V.,2025-12-08 00:00:00,Гуманітарна допомога,17,2024-02-23,вінницька,ФОП,мельник+вінницька
818,3302705015,ФОП Мельник,221100223A,АКДП,"DTP VACCINE (2023) Adsorbed Diphtheria, Tetanu...","СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2025-05-31 00:00:00,Гуманітарна допомога,40,2024-02-23,вінницька,ФОП,мельник+вінницька
819,3302705015,ФОП Мельник,X007042,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26 00:00:00,Гуманітарна допомога,39,2024-02-23,вінницька,ФОП,мельник+вінницька
820,3302705015,ФОП Мельник,UFA23503,ГепВ,ЕУВАКС В ВАКЦИНА ДЛЯ ПРОФІЛАКТИКИ ГЕПАТИТУ В Р...,"ЕлДжі Лайф Сайенсіс Лтд,Корея",2026-03-07 00:00:00,Державний бюджет 2022,50,2024-02-23,вінницька,ФОП,мельник+вінницька
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18385,3016503465,ФОП Мирошниченко,0401622,АДП,"АДСОРБОВАНА ВАКЦИНА ДТ / ADSORBED DT VACCINE, ...","ПТ БІО ФАРМА (Персеро), Індонезія",2025-12-21 00:00:00,Державний бюджет 2021,10,2024-02-23,запорізька,ФОП,мирошниченко+запорізька
18386,3016503465,ФОП Мирошниченко,2310002A,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V. Netherlands,2026-02-14 00:00:00,Гуманітарна допомога,5,2024-02-23,запорізька,ФОП,мирошниченко+запорізька
18387,3016503465,ФОП Мирошниченко,2863X026B,Пента,"Diphtheria, Tetanus, WholeCell Pertussis, Hepa...","СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2026-01-31 00:00:00,Державний бюджет 2022,10,2024-02-23,запорізька,ФОП,мирошниченко+запорізька
18388,3016503465,ФОП Мирошниченко,68D23040А,ОПВ,"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bharat Biotech Int.Ltd, Індія",2024-08-01 00:00:00,Гуманітарна допомога,10,2024-02-23,запорізька,ФОП,мирошниченко+запорізька


In [16]:
fops_test_df = test_df[test_df['legal_entity_edrpou'] == 'ФОП']
fops_test_df['Прізвище'] = fops_test_df['legal_entity_name'].str.split(' ').str[0].str.lower() + '+' + fops_test_df['registration_area'].str.lower()
fops_test_df

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_9876/769667654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fops_test_df['Прізвище'] = fops_test_df['legal_entity_name'].str.split(' ').str[0].str.lower() + '+' + fops_test_df['registration_area'].str.lower()


,legal_entity_id,legal_entity_name,legal_entity_edrpou,care_type,property_type,legal_entity_email,legal_entity_website,legal_entity_phone,legal_entity_owner_name,registration_area,registration_settlement,registration_address,lat,lng,Прізвище
4,a2e144ce-0307-4c6e-bdb9-6186d519a531,МІСЬКО ВІТА ВАСИЛІВНА,ФОП,Первинна,ФОП,vt.klinik@gmail.com,NaN,+380506906522,Місько Віта Василівна,ЧЕРНІВЕЦЬКА,ЗАСТАВНА,"Україна, 59400, Чернівецька обл., Заставнівськ...",48.531439,25.833238,місько+чернівецька
6,eb1b47d6-d116-446f-bfd4-d000884574ca,ЯКИМЧУК НАТАЛІЯ ВАЛЕРІЇВНА,ФОП,Первинна,ФОП,n.yakimchuk@homedoctor.in.ua,homedoctor.in.ua,+380637000640,Якимчук Наталія Валеріївна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.578154,30.799197,якимчук+одеська
8,d0549868-32e3-4c83-88f3-dd938f984b15,СЕЛЕЗНЬОВА СВІТЛАНА ЛЕОНІДІВНА,ФОП,Первинна,ФОП,medkabinet43@gmail.com,NaN,+380980632119,Селезньова Світлана Леонідівна,ОДЕСЬКА,БІЛГОРОД-ДНІСТРОВСЬКИЙ,"Україна, 67700, Одеська обл., місто Білгород-Д...",46.190439,30.342594,селезньова+одеська
9,760c6b14-64ea-4886-92d6-7468860e6a63,ДОЛІШНА ОКСАНА ІВАНІВНА,ФОП,Первинна,ФОП,doleshniy.andriy@gmail.com,NaN,+380995113771,Долішна Оксана Іванівна,ІВАНО-ФРАНКІВСЬКА,ІВАНО-ФРАНКІВСЬК(З),"Україна, 76018, Івано-Франківська обл., місто ...",48.919247,24.722991,долішна+івано-франківська
10,e61e9f53-3bf8-4b33-bf49-eb37a9c84bec,ЮРЧЕНКО ОКСАНА ВОЛОДИМИРІВНА,ФОП,Первинна,ФОП,dr.oksanayurchenko@gmail.com,NaN,+380992860263,Юрченко Оксана Володимирівна,ПОЛТАВСЬКА,ЛОХВИЦЯ,"Україна, 37200, Полтавська обл., Миргородський...",50.358872,33.259035,юрченко+полтавська
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4094,303371f3-c58e-4394-bc7d-734970961a61,РАДЬКО ІННА ОЛЕКСАНДРІВНА,ФОП,Первинна,ФОП,innaradko84@gmail.com,NaN,+380991239202,Радько Інна Олександрівна,КІРОВОГРАДСЬКА,КРОПИВНИЦЬКИЙ,"Україна, 25031, Кіровоградська обл., місто Кро...",48.507933,32.262317,радько+кіровоградська
4095,6378e55a-4ce1-4ec1-9eb6-0e73b2ecbe56,Філатов Олександр Володимирович,ФОП,Первинна,ФОП,filatovav.home@gmail.com,NaN,+380932463629,Філатов Олександр Володимирович,КИЇВСЬКА,СЛАВУТИЧ(З),"Україна, 07101, Київська обл., місто Славутич(...",51.518644,30.760449,філатов+київська
4111,a9e06381-77e1-4ce3-b66a-c646b1189e8b,ШАРЕНКО-ЛАПТЄВА ЛАРИСА ВОЛОДИМИРІВНА,ФОП,Первинна,ФОП,sharenko.fop@gmail.com,NaN,+380501720659,Шаренко-Лаптєва Лариса Володимирівна,ПОЛТАВСЬКА,ПИРЯТИН,"Україна, 37000, Полтавська обл., Пирятинський ...",50.251287,32.506312,шаренко-лаптєва+полтавська
4116,5c6d9036-7a55-40de-aa86-b5c355303bbb,ПАВЛЕНКО НАТАЛІЯ ОЛЕКСІЇВНА,ФОП,Первинна,ФОП,nata.pa1976@gmail.com,NaN,+380956281094,Павленко Наталія Олексіївна,ХАРКІВСЬКА,БОРОВА,"Україна, 63801, Харківська обл., Ізюмський р-н...",49.374251,37.623014,павленко+харківська


In [17]:
fops_test_df['registration_area'].value_counts()

registration_area
ОДЕСЬКА              100
ДНІПРОПЕТРОВСЬКА      87
ЛЬВІВСЬКА             71
М.КИЇВ                50
РІВНЕНСЬКА            46
ХАРКІВСЬКА            43
ЗАКАРПАТСЬКА          42
ІВАНО-ФРАНКІВСЬКА     41
КИЇВСЬКА              38
ВІННИЦЬКА             38
КІРОВОГРАДСЬКА        33
ХМЕЛЬНИЦЬКА           32
ПОЛТАВСЬКА            28
ЗАПОРІЗЬКА            23
СУМСЬКА               23
ВОЛИНСЬКА             22
ТЕРНОПІЛЬСЬКА         17
ХЕРСОНСЬКА            17
ЧЕРКАСЬКА             15
ЧЕРНІГІВСЬКА          14
ЖИТОМИРСЬКА           14
ЧЕРНІВЕЦЬКА            9
МИКОЛАЇВСЬКА           9
ДОНЕЦЬКА               4
ЛУГАНСЬКА              3
Name: count, dtype: int64

##### Звіт по кількості закладів, які мають ПМД і містяться в таблиці ЦГЗ:

In [28]:
print(f"""1. Всього закладів у таблиці ЦГЗ – {df['код ЄДРПОУ'].unique().shape[0]} 
    Закладів – {df[['код ЄДРПОУ', 'property_type']].drop_duplicates()['property_type'].value_counts().loc['Заклад']}
    ФОП – {df[['код ЄДРПОУ', 'property_type']].drop_duplicates()['property_type'].value_counts().loc["ФОП"]}""")

print(f"""2. ⁠Всього закладів у пакеті ПМД НСЗУ – {test_df['legal_entity_id'].unique().shape[0]} 
    Закладів – {test_df[test_df['property_type']!='ФОП']['legal_entity_id'].unique().shape[0]}
    ФОП – {test_df[test_df['property_type']=='ФОП']['legal_entity_id'].unique().shape[0]}""")

facility_codes = df[df['property_type'] == 'Заклад']['код ЄДРПОУ'].dropna().unique()
facilities_reporting = set(facility_codes) & test_edrdou_set
fops_reporting = set(fops_df['Прізвище'].dropna().unique()) & set(fops_test_df['Прізвище'].unique())
print(f"""3. ⁠Скільки закладів з договором ПМД звітують у табличці ЦГЗ – {len(facilities_reporting) + len(fops_reporting)} 
    Закладів – {len(facilities_reporting)}
    ФОП – {len(fops_reporting)}""")

facilities_reporting_but_absent = set(facility_codes) - test_edrdou_set
fops_reporting_but_absent = set(fops_df['Прізвище'].unique()) - set(fops_test_df['Прізвище'].unique())
print(f"""4. ⁠Скільки звітують у табличці ЦГЗ, але не мають договору ПМД – {len(facilities_reporting_but_absent) + len(fops_reporting_but_absent)} 
    Закладів – {len(facilities_reporting_but_absent)}
    ФОП – {len(fops_reporting_but_absent)}""")

1. Всього закладів у таблиці ЦГЗ – 1876 
    Закладів – 1531
    ФОП – 346
2. ⁠Всього закладів у пакеті ПМД НСЗУ – 2479 
    Закладів – 1660
    ФОП – 819
3. ⁠Скільки закладів з договором ПМД звітують у табличці ЦГЗ – 1400 
    Закладів – 1064
    ФОП – 336
4. ⁠Скільки звітують у табличці ЦГЗ, але не мають договору ПМД – 506 
    Закладів – 467
    ФОП – 39


In [ ]:
len(set(fops_df['Прізвище'].unique()) - set(fops_test_df['Прізвище'].unique()))

39

In [ ]:
r1 = fops_df[fops_df['Прізвище'].isin(set(fops_df['Прізвище'].unique()) - set(fops_test_df['Прізвище'].unique()))].drop_duplicates('код ЄДРПОУ')
r1.to_excel('Results/ФОПи, які звітують по залишкам, але відсутні в таблиці ЦГЗ.xlsx', index=False)
r1

,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Торгівельна назва,Виробник,Термін придатності,Джерело фінансування,Кількість доз,дата оновлення,Регіон,property_type,Прізвище
936,3255920623,ФОП Григоришина,1141M019A,ХІБ,ВАКЦИНА КОН`ЮГОВАНА ДЛЯ ПРОФІЛАКТИКИ ЗАХВОРЮВА...,"СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",2024-05-21 00:00:00,Державний бюджет 2021,11,2024-02-23,Вінницька область,ФОП,григоришина
963,2335110866,ФОП Червякова,UFA22005,ГепВ,"ЕУВАКС В Вакцина для профілактики гепатиту В, ...","ЕлДжі Лайф Сайенсіс Лтд,Корея",2025-03-10 00:00:00,Державний бюджет 2021,5,2024-02-23,Вінницька область,ФОП,червякова
1384,2673513087,ФОП Г.Кліщевська,X007788,КПК,"MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",2024-05-26 00:00:00,Гуманітарна допомога,34,2024-02-23,Волинська область,ФОП,г.кліщевська
1618,3166913315,ФОП Михалевський Б.,0401622,АДП,"АДСОРБОВАНА ВАКЦИНА ДТ / ADSORBED DT VACCINE, ...","ПТ БІО ФАРМА (Персеро), Індонезія",2025-12-21 00:00:00,Державний бюджет 2021,19,2024-02-23,Волинська область,ФОП,михалевський
3052,2733508291,"ФОП ""Горошко А.М.""",D2720-E1,АДП-М,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",ББ-НЦІПХЛтд. Болгарія,2026-02-28 00:00:00,Державний бюджет 2021,46,2024-02-23,Дніпропетровська область,ФОП,"""горошко"
3081,2889703358,"ФОП ""Шпорт А.Я.""",FWL21506,Пента,"Eupenta inj.DTP-Hib-HepB 0,5mL","ЕлДжі Лайф Сайенсіс Лтд,Корея",2024-12-31 00:00:00,Гуманітарна допомога,48,2024-02-23,Дніпропетровська область,ФОП,"""шпорт"
3390,2986403644,ФОП Климук К.В.,68D23040A,ОПВ,"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bharat Biotech Int.Ltd, Індія",2024-08-01 00:00:00,Гуманітарна допомога,10,2024-02-23,Дніпропетровська область,ФОП,климук
3820,2562701239,"ФОП ""Маланчик""",2249001B,ІПВ,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V.,2025-12-08 00:00:00,Гуманітарна допомога,11,2024-02-16,Донецька область,ФОП,"""маланчик"""
4560,2584015708,ФОП Венгер С.І,2204008A,ІПВ,Inactivated poliomyelitis vaccine,Bilthoven Biologicals B.V.A. Netherlands,2024-12-20 00:00:00,Гуманітарна допомога,20,2024-02-23,Житомирська область,ФОП,венгер
5253,43968084,"ФОП ""Бенько Г.С.""",D2708,АДП-М,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...","ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",2025-11-30 00:00:00,Державний бюджет 2021,17,2024-02-09,Івано-Франківська область,ФОП,"""бенько"


In [ ]:
len(set(fops_test_df['Прізвище'].unique()) - set(fops_df['Прізвище'].unique()))

465

In [ ]:
r2 = fops_test_df[fops_test_df['Прізвище'].isin(set(fops_test_df['Прізвище'].unique()) - set(fops_df['Прізвище'].unique()))]
r2.to_excel('Results/ФОПи, присутні в таблиці ЦГЗ, але не звітують по залишкам.xlsx', index=False)
r2

,legal_entity_id,legal_entity_name,legal_entity_edrpou,care_type,property_type,legal_entity_email,legal_entity_website,legal_entity_phone,legal_entity_owner_name,registration_area,registration_settlement,registration_address,lat,lng,Прізвище
6,eb1b47d6-d116-446f-bfd4-d000884574ca,ЯКИМЧУК НАТАЛІЯ ВАЛЕРІЇВНА,ФОП,Первинна,ФОП,n.yakimchuk@homedoctor.in.ua,homedoctor.in.ua,+380637000640,Якимчук Наталія Валеріївна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.578154,30.799197,якимчук
9,760c6b14-64ea-4886-92d6-7468860e6a63,ДОЛІШНА ОКСАНА ІВАНІВНА,ФОП,Первинна,ФОП,doleshniy.andriy@gmail.com,NaN,+380995113771,Долішна Оксана Іванівна,ІВАНО-ФРАНКІВСЬКА,ІВАНО-ФРАНКІВСЬК(З),"Україна, 76018, Івано-Франківська обл., місто ...",48.919247,24.722991,долішна
12,701bc242-68f9-4773-97e2-da36e2561c31,АРУТЮНЯН АНУШ ГНЕЛІВНА,ФОП,Первинна,ФОП,anush@homedoctor.in.ua,homedoctor.in.ua,+380734444777,Арутюнян Ануш Гнелівна,ОДЕСЬКА,ОДЕСА,"Україна, 65000, Одеська обл., місто Одеса, ВУЛ...",46.489375,30.727988,арутюнян
18,66a40717-83ec-4312-ac1e-cac5b4fd0786,ПОДПЕРИГОРА ІННА ВАСИЛІВНА,ФОП,Первинна,ФОП,i.podperygora@homedoctor.in.ua,https://homedoctor.in.ua,+380963955669,ПОДПЕРИГОРА ІННА ВАСИЛІВНА,ОДЕСЬКА,ОДЕСА,"Україна, 65020, Одеська обл., місто Одеса, вул...",46.476420,30.740399,подперигора
19,47933c79-60dc-4a69-99f6-b96c31bed482,ВИШНЕВСЬКИЙ ОЛЕКСАНДР ОЛЕКСАНДРОВИЧ,ФОП,Первинна,ФОП,nashlekarz@gmail.com,NaN,+380987798456,Вишневський Олександр Олександрович,ЖИТОМИРСЬКА,ОЛЕВСЬК,"Україна, 11001, Житомирська обл., Коростенськи...",51.215174,27.641198,вишневський
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,f0a7ea1e-c038-49b4-9ce2-edceb8ea499d,Добрянський Олесь Степанович,ФОП,Первинна,ФОП,office@stepmed.com.ua,https://stepmed.com.ua,+380981680088,Добрянський Олесь Степанович,ЛЬВІВСЬКА,ПУСТОМИТИ(З),"Україна, 81100, Львівська обл., Пустомитівськи...",NaN,NaN,добрянський
4030,723a4b71-fe91-4461-a745-64b62951b5ce,УЛІТІНА ОЛЕНА ІВАНІВНА,ФОП,Первинна,ФОП,112ulit@gmail.com,NaN,+380669763020,Улітіна Олена Іванівна,ХЕРСОНСЬКА,НОВА КАХОВКА(З),"Україна, 74900, Херсонська обл., місто Нова Ка...",46.796690,33.470869,улітіна
4042,5258a1ba-01ed-4354-afd4-449ca3666b28,СЕМКО МАР’ЯНА БОГДАНІВНА,ФОП,Первинна,ФОП,semkomariana@gmail.com,NaN,+380978747112,Семко Мар'яна Богданівна,ІВАНО-ФРАНКІВСЬКА,ПЕРЕГІНСЬКЕ,"Україна, 77662, Івано-Франківська обл., Рожнят...",48.813750,24.193254,семко
4079,3d12fe6f-0715-4a9c-9e33-9ab1b72b040f,Кравець Анатолій Вікторович,ФОП,Первинна,ФОП,a.kravets73@gmail.com,NaN,+380633003410,Кравець Анатолій Вікторович,М.КИЇВ,КИЇВ,"Україна, 02140, місто Київ, вул.Гмирі Бориса, ...",NaN,NaN,кравець


#### Створити унікальні коди для кожного ФОП, використовуючи файл даних з уникальними назвами 

##### Вивести відповідність ЄДРПОУ і назви закладу

In [ ]:
import pandas as pd

out_df = test_df[['legal_entity_edrpou', 'legal_entity_name']]
out_df = out_df[out_df['legal_entity_edrpou']!='ФОП']
out_df

,legal_entity_edrpou,legal_entity_name
2,38617509,КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО УСАТІВСЬК...
3,37934309,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ..."
5,37691162,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""МЕДИЧ..."
15,41774140,"КОМУНАЛЬНЕ ПІДПРИЄМСТВО ""МЕДИЧНИЙ ЛІКУВАЛЬНО-Д..."
22,40393866,"""КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЛУКІВС..."
...,...,...
4105,41100920,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""100% ..."
4106,42896454,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""АМБУЛАТО..."
4114,37755220,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ДОБРОПІЛ..."
4115,38458175,КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО МИКОЛАЇВ...


In [ ]:
df_for_fops = pd.read_csv('Data/Auxillary/Дані для співставлення .csv')
df_for_fops

,код ЄДРПОУ,Заклад,Серія препарту,Міжнародна непатентована назва,Торгівельна назва,Виробник,Термін придатності,Джерело фінансування,Кількість доз,дата оновлення,Регіон,Назва закладу (автоматична)
0,3.552733e+07,"КНП ""ЦПМСД №1"" м. Вінниця",1141M019A,Вакцина для профілактики гемофільної інфекції ...,ВАКЦИНА КОН`ЮГОВАНА ДЛЯ ПРОФІЛАКТИКИ ЗАХВОРЮВА...,"СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",45433.0,Державний бюджет 2021,54,45345.0,Вінницька область,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ..."
1,3.552733e+07,"КНП ""ЦПМСД №1"" м. Вінниця",X007788,"Вакцина для профілактики кору, паротиту та кра...","MMRVAXPRO II 0,5ML 10X1DOSE VIAL L25","Merck & Co., Inc.",45438.0,Гуманітарна допомога,6,45345.0,Вінницька область,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ..."
2,3.552733e+07,"КНП ""ЦПМСД №1"" м. Вінниця",D2708,Анатоксин для профілактики дифтерії та правцю ...,"ТЕТАДІФ суспензія для ін'єкцій 0,5 мл (1 доза)...",ББ-НЦІПХЛтд. Болгарія,45991.0,Державний бюджет 2021,113,45345.0,Вінницька область,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ..."
3,3.552733e+07,"КНП ""ЦПМСД №1"" м. Вінниця",2249001B,Вакцина для профілактики поліомієліту (інактив...,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V.,45999.0,Гуманітарна допомога,13,45345.0,Вінницька область,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ..."
4,3.552733e+07,"КНП ""ЦПМСД №1"" м. Вінниця",62С22009А,Вакцина для профілактики сказу,"ІНДІРАБ ВАКЦИНА АНТИРАБІЧНА ОЧИЩЕНА, ІНАКТИВОВ...","ТОВ ""Фарма Лайф"", Україна",45961.0,Державний бюджет 2021,7,45345.0,Вінницька область,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
18480,3.016503e+09,ФОП Мирошниченко,2310002A,Вакцина для профілактики поліомієліту (інактив...,Inactivated Poliomyelitis Vaccine,Bilthoven Biologicals B.V. Netherlands,46067.0,Гуманітарна допомога,5,45345.0,Запорізька область,ФОП
18481,3.016503e+09,ФОП Мирошниченко,2863X026B,Комбінована вакцина для профілактики кашлюка з...,"Diphtheria, Tetanus, WholeCell Pertussis, Hepa...","СЕРУМ ІНСТИТУТ ІНДІЇ ПВТ. ЛТД., Індія",46053.0,Державний бюджет 2022,10,45345.0,Запорізька область,ФОП
18482,3.016503e+09,ФОП Мирошниченко,68D23040А,Вакцина для профілактики поліомієліту (оральна),"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bharat Biotech Int.Ltd, Індія",45505.0,Гуманітарна допомога,10,45345.0,Запорізька область,ФОП
18483,3.873260e+07,"ТОВ ""ОНКОЛАЙФ""",68D23040А,Вакцина для профілактики поліомієліту (оральна),"Bivalent Poliomyelitis Vaccine Tipe 1&3, Live ...","Bharat Biotech Int.Ltd, Індія",45505.0,Гуманітарна допомога,20,45345.0,Запорізька область,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ОНКОЛ..."


In [ ]:
fop_records = df_for_fops[df_for_fops["Назва закладу (автоматична)"] == 'ФОП'].groupby([' код ЄДРПОУ']).apply(lambda x: x[' Заклад'].unique()[0]).reset_index()
fop_records

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_56653/476761345.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fop_records = df_for_fops[df_for_fops["Назва закладу (автоматична)"] == 'ФОП'].groupby([' код ЄДРПОУ']).apply(lambda x: x[' Заклад'].unique()[0]).reset_index()


,код ЄДРПОУ,0
0,3.654483e+07,ФОП Кравчук
1,4.064779e+07,ФОП Кошелюк
2,4.064780e+07,ФОП Кошелюк
3,4.064780e+07,ФОП Кошелюк
4,4.064780e+07,ФОП Кошелюк
...,...,...
340,3.565603e+09,ФОП Бєлова
341,3.567513e+09,ФОП Кузьмін В.М.
342,3.609606e+09,ФОП Лозова
343,3.679408e+09,ФОП Демчук Ольга Ігорівна


In [ ]:
# out_df.drop_duplicates().to_excel('Data/Auxillary/Відповідність ЄДРПОУ і назви.xlsx', sheet_name='Відповідники', index=False)
# out_df = pd.concat([out_df, fop_records.rename(columns={'код ЄДРПОУ':'legal_entity_edrpou', 0: 'legal_entity_name'})], axis=0)
# out_df
out_df = pd.concat([out_df.rename(columns={'legal_entity_edrpou':'код ЄДРПОУ', 'legal_entity_name': 'Назва закладу'}), 
                    fop_records.rename(columns={' код ЄДРПОУ':'код ЄДРПОУ', 0: 'Назва закладу'})], axis=0)

In [ ]:
out_df['код ЄДРПОУ'] = out_df['код ЄДРПОУ'].astype(int)
out_df.to_excel('Data/Auxillary/Відповідність ЄДРПОУ і назви.xlsx', sheet_name='Відповідники', index=False)